In [ ]:
!pip install streamlit requests


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 795.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.9 MB/s eta 0:00:00


In [ ]:
!pip install  pyngrok


In [ ]:
%%writefile streamlit_app.py

import streamlit as st
import requests

# Ngrok URL from FastAPI
NGROK_URL = "https://fdad-34-125-184-245.ngrok-free.app/"  # Replace with your Ngrok URL

# Streamlit app
st.title("AI Assistant with Voice and Text Input")

# Input method selection
input_method = st.radio("Choose input method:", ("Type", "Voice"))

if input_method == "Type":
    query = st.text_input("What would you like to know about?")
    if st.button("Search and Summarize"):
        response = requests.post(f"{NGROK_URL}/search_and_summarize", json={"query": query})
        if response.status_code == 200:
            st.write("### Summary:")
            st.write(response.json()["summary"])
        else:
            st.error(f"Error: {response.json().get('detail', 'Unknown error')}")

elif input_method == "Voice":
    audio_file = st.file_uploader("Upload an audio file", type=["wav", "mp3"])
    if audio_file and st.button("Transcribe and Summarize"):
        try:
            # Notify the user that the file has been received
            st.info("File received. Processing...")

            # Send the audio file to the FastAPI backend
            files = {"file": audio_file}
            response = requests.post(f"{NGROK_URL}/transcribe_audio", files=files)

            if response.status_code == 200:
                transcription = response.json()["transcription"]
                st.success("Audio file processed successfully!")
                st.write("### Transcription:")
                st.write(transcription)

                # Use the transcription as a query for summarization
                summary_response = requests.post(f"{NGROK_URL}/search_and_summarize", json={"query": transcription})
                if summary_response.status_code == 200:
                    st.write("### Summary:")
                    st.write(summary_response.json()["summary"])
                else:
                    st.error(f"Error: {summary_response.json().get('detail', 'Unknown error')}")
            else:
                st.error(f"Error: {response.json().get('detail', 'Unknown error')}")
        except Exception as e:
            st.error(f"Error uploading file: {e}")

Overwriting streamlit_app.py


In [ ]:
!ngrok authtoken 2shFBySTClb5xE6aeZ8rpkjhu0A_7FuVC2vsaSKVCiv1Y3Vta


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:

from pyngrok import ngrok

# Kill any existing tunnels
ngrok.kill()

# Start a new tunnel for Streamlit
public_url = ngrok.connect(8501)
print(f"🚀 Streamlit Public URL: {public_url}")

# Run Streamlit
!streamlit run streamlit_app.py &>/dev/null &

🚀 Streamlit Public URL: NgrokTunnel: "https://3e76-34-106-49-2.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
!curl -X POST "https://0e0e-34-125-184-245.ngrok-free.app/search_and_summarize" \
-H "Content-Type: application/json" \
-d '{"query": "Give me latest news on cake?"}'

{"summary":"CAKE was selected as a 2023 Breakthrough Brand by Interbrand Index. CAKE has returned 15.9% so far this year. The Retail-Wholesale sector has returned 6.9%."}